# Washington State DOT Traffic API

source: [https://wsdot.wa.gov/traffic/api](https://wsdot.wa.gov/traffic/api)


In [1]:
import pandas as pd
import json
import os
import requests

from dotenv import load_dotenv
from pathlib import Path
from sqlalchemy import create_engine

## Import env variables


In [2]:
# load the .env file
load_dotenv()

# API variables
api_key = os.getenv("API_KEY")

# MySQL database variables
db_user = os.getenv("AZURE_USERNAME")
db_pwd = os.getenv("AZURE_PWD")
db_host = os.getenv("AZURE_URL")
db_port = os.getenv("AZURE_PORT")
db_database = os.getenv("AZURE_DB")

# print(api_key)

## URLs to Access APIs


In [3]:
TRAVEL_TIMES_URL = "http://wsdot.wa.gov/Traffic/api/TravelTimes/TravelTimesREST.svc/GetTravelTimesAsJson?AccessCode={ACCESSCODE}"

TRAFFIC_ALERTS_URL = "http://www.wsdot.wa.gov/Traffic/api/HighwayAlerts/HighwayAlertsREST.svc/GetAlertsAsJson?AccessCode={ACCESSCODE}"

WEATHER_INFORMATION_URL = "http://wsdot.wa.gov/Traffic/api/WeatherInformation/WeatherInformationREST.svc/GetCurrentWeatherInformationAsJson?AccessCode={ACCESSCODE}"

### Custom function to get API data


In [4]:
def get_api_data(url, access_key):
    """_summary_

    Args:
        url (_type_): _description_
        access_key (_type_): _description_

    Returns:
        _type_: _description_
    """
    # create the url with the access key
    url_api = url.format(ACCESSCODE=access_key)
    response = requests.get(url_api)

    # check if request was successful
    if response.status_code == 200:
        print("Data fetched successfully.")
        data = response.json()
        df = pd.DataFrame(data)
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

    # return the dataframe and the status code
    return df, response.status_code

In [5]:
def convert_dict_to_json(dataframe):

    for col in dataframe.columns:
        if dataframe[col].apply(lambda x: isinstance(x, dict)).all():
            dataframe[col] = dataframe[col].apply(lambda x: json.dumps(x))

In [6]:
def get_updates(
    travel_times_url, traffic_alerts_url, weather_information_url, access_key
):

    # Get current time
    timestamp = pd.Timestamp.now()
    df_tt, response_tt = get_api_data(travel_times_url, access_key)
    df_ta, response_ta = get_api_data(traffic_alerts_url, access_key)
    df_wa, response_wa = get_api_data(weather_information_url, access_key)

    # add pull_date_time
    df_tt["timestamp"] = timestamp
    df_ta["timestamp"] = timestamp
    df_wa["timestamp"] = timestamp

    # convert dict columns to json
    convert_dict_to_json(df_tt)
    convert_dict_to_json(df_ta)
    convert_dict_to_json(df_wa)

    df_api_fetch = pd.DataFrame(
        [
            {
                "timestamp": timestamp,
                "travel_times_response": response_tt,
                "traffic_alerts_response": response_ta,
                "weather_alerts_response": response_wa,
            }
        ]
    )

    return df_api_fetch, df_tt, df_ta, df_wa

# EXTRACT

## Call to get all info from API


In [7]:
df_api_fetch, df_tt, df_ta, df_wa = get_updates(
    TRAVEL_TIMES_URL, TRAFFIC_ALERTS_URL, WEATHER_INFORMATION_URL, api_key
)

Data fetched successfully.
Data fetched successfully.
Data fetched successfully.


# LOAD


## Connection to MySQL Database


In [8]:
# Connect to database
connection_url = (
    f"mysql+pymysql://{db_user}:{db_pwd}" f"@{db_host}:{db_port}/{db_database}"
)
try:
    engine = create_engine(connection_url)
    print("Connection successful!")
except Exception as e:
    print("Connection unsuccessful! The error is: ", e)

Connection successful!


## Upload Data to MySQL


In [13]:
def upload_to_db(dataframe, table_name, conn):
    """_summary_

    Args:
        dataframe (_type_): _description_
        table_name (_type_): _description_
    """
    try:
        dataframe.to_sql(
            table_name,
            con=conn,
            if_exists="replace",
            index=False,
        )
        print(f"{table_name} uploaded successfully!")
    except Exception as e:
        print(f"Failed to upload {table_name}! The error is: ", e)

In [10]:
# # Table api_fetch
# df_api_fetch.to_sql("api_fetch", con=engine, if_exists="append", index=False)

# # Table time_trave_raw
# df_tt.to_sql("time_travel_raw", con=engine, if_exists="append", index=False)

# # Table traffic_alerts_raw
# df_ta.to_sql("traffic_alerts_raw", con=engine, if_exists="append", index=False)

# # Table weather_alerts_raw
# df_wa.to_sql("weather_alerts_raw", con=engine, if_exists="append", index=False)

# print("Data saved to database.")

In [14]:
def extract_and_load():
    # Get data from the API
    df_api_fetch, df_tt, df_ta, df_wa = get_updates(
        TRAVEL_TIMES_URL, TRAFFIC_ALERTS_URL, WEATHER_INFORMATION_URL, api_key
    )

    # Connect to database
    connection_url = (
        f"mysql+pymysql://{db_user}:{db_pwd}" f"@{db_host}:{db_port}/{db_database}"
    )

    try:
        engine = create_engine(connection_url)
        with engine.connect() as conn:
            print("Connection successful!")

            # Upload data to the database
            upload_to_db(df_api_fetch, "api_fetch", conn)
            upload_to_db(df_tt, "time_travel_raw", conn)
            upload_to_db(df_ta, "traffic_alerts_raw", conn)
            upload_to_db(df_wa, "weather_alerts_raw", conn)

    except Exception as e:
        print("Connection unsuccessful! The error is: ", e)

In [15]:
extract_and_load()

Data fetched successfully.
Data fetched successfully.
Data fetched successfully.
Connection successful!
api_fetch uploaded successfully!
time_travel_raw uploaded successfully!
traffic_alerts_raw uploaded successfully!
weather_alerts_raw uploaded successfully!


In [16]:
import schedule
import time

schedule.every(5).minutes.do(extract_and_load)

while True:
    schedule.run_pending()
    time.sleep(60)

Data fetched successfully.
Data fetched successfully.
Data fetched successfully.
Connection successful!
api_fetch uploaded successfully!
time_travel_raw uploaded successfully!
traffic_alerts_raw uploaded successfully!
weather_alerts_raw uploaded successfully!
Data fetched successfully.
Data fetched successfully.
Data fetched successfully.
Connection successful!
api_fetch uploaded successfully!
time_travel_raw uploaded successfully!
traffic_alerts_raw uploaded successfully!
weather_alerts_raw uploaded successfully!
Data fetched successfully.
Data fetched successfully.
Data fetched successfully.
Connection successful!
api_fetch uploaded successfully!
time_travel_raw uploaded successfully!
traffic_alerts_raw uploaded successfully!
weather_alerts_raw uploaded successfully!
Data fetched successfully.
Data fetched successfully.
Data fetched successfully.
Connection successful!
api_fetch uploaded successfully!
time_travel_raw uploaded successfully!
traffic_alerts_raw uploaded successfully!
we

KeyboardInterrupt: 

In [18]:
df_check = pd.read_sql("SELECT * FROM time_travel_raw", engine)
display(df_check)

,id,averagetime,description,currenttime,distance,endpoint,name,startpoint,timeupdated,traveltimeid,timestamp
0,1009,41.0,Everett to Downtown Seattle using HOV lanes,31.0,26.72,"{""Latitude"": 47.609294, ""MilePost"": 165.83, ""R...",Everett-Seattle HOV,"{""Latitude"": 47.964146, ""MilePost"": 192.55, ""R...",/Date(1738714800000-0800)/,2,2025-02-04 16:23:32
1,1010,48.0,Downtown Seattle to Everett using HOV lanes,28.0,26.94,"{""Latitude"": 47.92428, ""MilePost"": 192.77, ""Ro...",Seattle-Everett HOV,"{""Latitude"": 47.609294, ""MilePost"": 165.83, ""R...",/Date(1738714800000-0800)/,3,2025-02-04 16:23:32
2,1011,63.0,Downtown Seattle to Everett,32.0,26.94,"{""Latitude"": 47.92428, ""MilePost"": 192.77, ""Ro...",Seattle-Everett,"{""Latitude"": 47.609294, ""MilePost"": 165.83, ""R...",/Date(1738714800000-0800)/,4,2025-02-04 16:23:32
3,1012,18.0,Downtown Bellevue to Issaquah,9.0,9.28,"{""Latitude"": 47.541799, ""MilePost"": 16.96, ""Ro...",Bellevue-Issaquah,"{""Latitude"": 47.6138, ""MilePost"": 13.33, ""Road...",/Date(1738714800000-0800)/,5,2025-02-04 16:23:32
4,1013,13.0,Downtown Bellevue to Issaquah using HOV lanes,10.0,9.28,"{""Latitude"": 47.541799, ""MilePost"": 16.96, ""Ro...",Bellevue-Issaquah HOV,"{""Latitude"": 47.6138, ""MilePost"": 13.33, ""Road...",/Date(1738714800000-0800)/,6,2025-02-04 16:23:32
...,...,...,...,...,...,...,...,...,...,...,...
6547,7556,8.0,SR500 WB and 112th Ave to SR500 and I-5 JCT,5.0,4.80,"{""Latitude"": 45.650552, ""MilePost"": 0.66, ""Roa...",021 SR500 WB and 112th Ave to SR500 and I-5 JCT,"{""Latitude"": 45.66206, ""MilePost"": 5.46, ""Road...",/Date(1738723500000-0800)/,528,2025-02-04 18:50:53
6548,7557,19.0,SR500 WB and 112th Ave to I-5 to I-5 at I-84 JCT,19.0,14.16,"{""Latitude"": 45.54485, ""MilePost"": 302.17, ""Ro...",021 SR500 WB and 112th Ave to I-5 to I-5 at I-...,"{""Latitude"": 45.66206, ""MilePost"": 5.46, ""Road...",/Date(1738723500000-0800)/,529,2025-02-04 18:50:53
6549,7558,20.0,SR500 WB and 112th Ave to I-205 to I-84 to I-5...,23.0,17.42,"{""Latitude"": 45.52976, ""MilePost"": 0.45, ""Road...",022 SR500 WB and 112th Ave to I-205 to I-84 to...,"{""Latitude"": 45.66206, ""MilePost"": 5.46, ""Road...",/Date(1738723500000-0800)/,530,2025-02-04 18:50:53
6550,7559,16.0,SB I-5 54th Ave To JBLM Main Gate,17.0,17.10,"{""Latitude"": 47.10493656, ""MilePost"": 121.1, ""...","SB I-5, Fife To JBLM Main Gate","{""Latitude"": 47.240632, ""MilePost"": 137.08, ""R...",/Date(1738723500000-0800)/,531,2025-02-04 18:50:53


In [19]:
df_schema = pd.read_sql("DESCRIBE weather_alerts_raw", engine)
display(df_schema)

,Field,Type,Null,Key,Default,Extra
0,id,int unsigned,NO,PRI,None,auto_increment
1,barometricpressure,float,YES,,None,
2,latitude,float,YES,,None,
3,longitude,float,YES,,None,
4,precipitationininches,float,YES,,None,
5,readingtime,varchar(45),YES,,None,
6,relativehumidity,float,YES,,None,
7,skycoverage,varchar(45),YES,,None,
8,stationid,varchar(45),YES,,None,
9,stationname,longtext,YES,,None,


In [44]:
cols = df_wa.columns
for col in cols:
    print(col)

BarometricPressure
Latitude
Longitude
PrecipitationInInches
ReadingTime
RelativeHumidity
SkyCoverage
StationID
StationName
TemperatureInFahrenheit
Visibility
WindDirection
WindDirectionCardinal
WindGustSpeedInMPH
WindSpeedInMPH
timestamp
wa_id
